In [ ]:
!python --version
!mkdir data

Python 3.10.12


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [6]:
from openai import OpenAI

In [7]:
from google.colab import userdata
key = userdata.get('openai') # Colab Secret

SecretNotFoundError: Secret openai does not exist.

In [9]:
with open("data/openai/quelle_est_bleue.txt") as f:
    text = f.read()

# print(text)

client = OpenAI(api_key=key)

completion = client.chat.completions.create(model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "De quoi parle ce texte? dans un contexte de plongeur pro"},
        {"role": "user", "content": text}
    ]
)

res = completion.choices[0].message
print(res.content)

NameError: name 'key' is not defined

In [ ]:
with open("data/openai/churn_cleaning.py") as f:
    text = f.read()

print(text)

client = OpenAI(api_key=key)

completion = client.chat.completions.create(model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Expliques moi ce que fais ce code"},
        {"role": "user", "content": text}
    ]
)

res = completion.choices[0].message
print(res.content)

import pandas as pd
import sweetviz
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None

df = pd.read_csv('data/churn/churn.csv', delimiter=',')
print(df.shape)

# Anonymisation
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis = 1)

# France only
df = df[df.Geography == "France"]
df = df.drop(["Geography"], axis = 1)
print(df.describe().T)

# my_report = sweetviz.analyze(df)
# my_report.show_html()

# Discretisation?
print(df.nunique())

# Etude pragmatique sur valeurs discretes (categorielles)
fig, axarr = plt.subplots(2, 2)
sns.countplot(x='Gender', hue = 'Exited',data = df, ax=axarr[0][0])
sns.countplot(x='HasCrCard', hue = 'Exited',data = df, ax=axarr[0][1])
sns.countplot(x='IsActiveMember', hue = 'Exited',data = df, ax=axarr[1][0])
plt.show()

# Etude pragmatique sur valeurs continues
fig, axarr = plt.subplots(3, 2)
sns.boxplot(y='CreditScore',x = 'Exited', hue = 'Exited',data = df, ax=axarr[0][0])
sns.boxplot(y='Age',x = 'Exited',

In [ ]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=b24b8cd3ed1b1bb824367d84ee69d45eea2e046cf87ab1bd8c428adea4d04237
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
import datetime
# import whisper # pip install openai-whisper
import os

In [ ]:
class OpenAIService:

    def __init__(self, whisper_model="base", chat_model="gpt-3.5-turbo-1106", chat_limit=16385):
        # os.environ["path"] += ";c:\\ffmpeg\\bin"
        # self.model = whisper.load_model(whisper_model)
        with open("data/openai/openai.env") as f:
            key = f.read()
        self.client = OpenAI(api_key=key)
        self.chat_model = chat_model
        self.chat_limit = chat_limit

    def mp3_to_text(self, path):
        result = self.model.transcribe(path)
        return result["text"].encode("utf-8").decode()

    def summary(self, text: str, nb: int=5) -> str:
        completion = self.client.chat.completions.create(
            model=self.chat_model,
            messages=[{"role": "system", "content": f"Fais moi un résumé en {nb} parties de ce texte"},
                {"role": "user", "content": text}])
        return completion.choices[0].message.content

    def correction(self, text: str, ponctuation=True, bank=True) -> str:
        s = "Corriges moi ce texte "
        if ponctuation:
            s+="avec de la ponctuation "
        if bank:
            s+="avec un vocabulaire bancaire"
        completion = self.client.chat.completions.create(
            model=self.chat_model,
            messages=[{"role": "system", "content": s},
                      {"role": "user", "content": text}])
        return completion.choices[0].message.content

    def translate(self, text: str, langue="anglais") -> str:
        s = f"Traduis moi ce texte en {langue}"
        completion = self.client.chat.completions.create(
            model=self.chat_model,
            messages=[{"role": "system", "content": s},
                      {"role": "user", "content": text}])
        return completion.choices[0].message.content

In [ ]:
print("OpenAI test")
openai = OpenAIService(whisper_model="base")
time0 = datetime.datetime.now()

# res = openai.mp3_to_text("data/openai/bank.mp3")
# print(res.strip())
# with open("data/openai/bank.txt", "w") as f:
#     f.write(res)

with open("data/openai/bank.txt", "r", encoding="iso-8859-1") as f:
    res = f.read()
correction = openai.correction(res, True, True)
print(correction)
with open(f"data/openai/correction.txt", "w") as f:
    f.write(correction)

OpenAI test


FileNotFoundError: [Errno 2] No such file or directory: 'data/openai/openai.env'

# Nouvelle section

In [ ]:
summary = openai.summary(res)
print(summary)
with open(f"data/openai/summary5.txt", "w") as f:
    f.write(summary)

1. Salimage, 32 ans, travaille comme conseiller clientèle à la Banque Postale, après avoir suivi une formation dispensée par La Poste et un centre de formation.

2. Il a évolué de conseiller financier à conseiller clientèle, proposant les produits et services financiers de la Banque Postale et gérant un portefeuille de 1600 clients, avec une attention particulière pour les 500 clients principaux.

3. Ses missions incluent la fourniture de conseils adaptés pour optimiser l'épargne et la constitution du patrimoine de ses clients, tout en développant le portefeuille à travers des actions sur les clients distanciers et de nouvelles approches client.

4. Son quotidien est marqué par la préparation des entretiens, le suivi des dossiers, les entretiens avec les clients, la gestion des appels et du service après-vente, ainsi que la veille sur l'actualité économique et financière.

5. Salimage trouve le contact avec la clientèle enrichissant et conseillerait à un jeune diplômé dans ce métier d'

In [ ]:
summary_english = openai.translate(summary)
print(summary_english)
with open(f"data/openai/summary_english.txt", "w") as f:
    f.write(summary_english)

1. Salimage, 32 years old, works as a customer advisor at La Banque Postale after completing training provided by La Poste and a training center.

2. He progressed from being a financial advisor to a customer advisor, offering La Banque Postale's financial products and services and managing a portfolio of 1600 clients, with a particular focus on 500 key clients.

3. His responsibilities include providing tailored advice to optimize his clients' savings and asset building, while also expanding the portfolio through actions targeting remote clients and new client approaches.

4. His daily routine involves preparing for meetings, tracking files, meeting with clients, handling calls and after-sales service, as well as staying updated on economic and financial news.

5. Salimage finds the client interaction enriching and would advise a young graduate in this profession to be persistent, enjoy challenges, be attentive and understanding, emphasizing the importance of advisory skills in this f

In [ ]:
summary_jpn = openai.translate(summary, "japonais")
print(summary_jpn)
with open(f"data/openai/summary_jpn.txt", "wb") as f:
    f.write(summary_jpn.encode("utf-8"))

1. サリマージュは32歳であり、郵便局でのトレーニングと研修センターでのトレーニングを受けた後、郵便局の顧客サービスアドバイザーとして働いています。

2. 彼は金融アドバイザーから顧客サービスアドバイザーに昇進し、郵便局の金融商品やサービスを提供し、1600人の顧客ポートフォリオを管理し、特に500人の主要顧客に重点を置いています。

3. 彼の任務には、顧客の貯蓄と資産の最適化のための適切なアドバイスの提供、遠隔地の顧客や新しいアプローチによるポートフォリオの開発が含まれています。

4. 彼の日常は面談の準備、案件のフォローアップ、顧客との面談、電話対応やアフターサービスの管理、そして経済・金融の最新情報に対する情報収集で印象づけられています。

5. サリマージュは顧客とのコンタクトを有益なものと感じ、この職業に就く若手の卒業生に、忍耐強く挑戦し、人々を聞き、理解すること、そしてこの分野でのアドバイスの重要性を強調しています。


In [ ]:
client = OpenAI(api_key=key)

response = client.images.generate(
  model="dall-e-3",
  prompt="La banque et Sopra Steria en 2030",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
print(image_url)


RateLimitError: Error code: 429 - {'error': {'code': 'rate_limit_exceeded', 'message': 'Rate limit exceeded for images per minute in organization org-HsnxNdW6VmuOpbzwkU3gojNp. Limit: 0/1min. Current: 1/1min. Please visit https://platform.openai.com/docs/guides/rate-limits to learn how to increase your rate limit.', 'param': None, 'type': 'requests'}}